# Synthetic Mock Demo

This notebook demonstrates how to:
- generate a synthetic 3D mock density + velocity field,
- load the resulting HDF5 file,
- compute the velocity magnitude,
- visualize a 2D slice.

This uses the `synthetic_mock` package.

In [19]:
from synthetic_mock.generators import generate_sample, RandomConfig

# Parameters
N = 128
L = 160.0

cfg = RandomConfig(
    num_attractors=3,
    num_repellers=2,
    seed=42
)

h5_path, specs = generate_sample(N=N, L=L, cfg=cfg, save_dir="output")

h5_path, specs

✅ Saved HDF5: output/N128_L160_A3_R2_seed42.hdf5


('output/N128_L160_A3_R2_seed42.hdf5',
 [AttractorSpec(x=43.832967768954134, y=-9.779449639671626, z=57.375667185821186, sigma=6.184208174356184, strength=0.6412660218314743, kind='attractor'),
  AttractorSpec(x=76.09957626188094, y=41.782352318456475, z=45.7702888443126, sigma=2.7686817960532752, strength=1.1755789068433506, kind='attractor'),
  AttractorSpec(x=-20.672316122787002, y=68.28239821577628, z=23.018419212906323, sigma=6.93656967962498, strength=1.1651212982409966, kind='attractor'),
  AttractorSpec(x=-43.6418045144357, y=8.733565922533572, z=-69.78923902333194, sigma=6.965787031955492, strength=1.4474965986830972, kind='repeller'),
  AttractorSpec(x=41.29403841365982, y=-23.27584509922106, z=75.31168390318453, sigma=7.3587267279331865, strength=1.6675752456106427, kind='repeller')])

In [20]:
import h5py
import numpy as np

with h5py.File(h5_path, "r") as f:
    d  = f["d"][:]
    vx = f["vx"][:]
    vy = f["vy"][:]
    vz = f["vz"][:]

d.shape, vx.shape, vy.shape, vz.shape

((128, 128, 128), (128, 128, 128), (128, 128, 128), (128, 128, 128))

In [27]:
import numpy as np
import pyvista as pv


def plot_velocity_field(vx, vy, vz, L, specs, skip=8, 
                        integration_direction="forward", 
                        point_size=14, label_font_size=18):
    """
    Visualize a 3D velocity field with streamlines and attractor labels 
    using a centered coordinate system [-L/2, L/2).

    Parameters
    ----------
    vx, vy, vz : ndarray
        3D velocity components, shape (N, N, N)
    L : float
        Physical box size
    specs : list of AttractorSpec
        Mock attractor/repeller specifications
    skip : int
        Subsampling factor for streamline seeds
    integration_direction : str
        'forward', 'backward', or 'both'
    """

    N = vx.shape[0]
    half = L / 2

    # ----------------------------------------
    # Build the PyVista grid
    # ----------------------------------------
    grid = pv.UniformGrid()
    grid.dimensions = vx.shape
    grid.spacing = (L / N, L / N, L / N)
    grid.origin = (-half, -half, -half)

    # velocity vectors
    vectors = np.stack([vx, vy, vz], axis=-1).reshape(-1, 3, order='F')
    grid.point_data["v"] = vectors

    # ----------------------------------------
    # Create streamline seed points
    # ----------------------------------------
    indices = np.indices(vx.shape).astype(float)
    coords = indices * (L / N) - half    # ✅ centered coords

    start_pts = coords[:, ::skip, ::skip, ::skip].reshape(3, -1).T

    # ----------------------------------------
    # Create plotter
    # ----------------------------------------
    plotter = pv.Plotter()
    plotter.show_bounds(bounds=[-half, half, -half, half, -half, half],
                        all_edges=True,
                        show_xlabels=True, show_ylabels=True, show_zlabels=True)
    plotter.add_mesh(grid.outline())

    # ----------------------------------------
    # Compute streamlines
    # ----------------------------------------
    stream = grid.streamlines_from_source(
        source=pv.wrap(start_pts),
        vectors="v",
        integrator_type=4,
        integration_direction=integration_direction,
        terminal_speed=0.0,
        max_time=400,
    )
    plotter.add_mesh(stream, opacity=0.15, color="white")

    # ----------------------------------------
    # Attractor / repeller markers + labels
    # ----------------------------------------
    for s in specs:
        pt = np.array([[s.x, s.y, s.z]])

        plotter.add_points(
            pt,
            render_points_as_spheres=True,
            point_size=point_size,
            color="cyan" if s.kind == "attractor" else "lime",
        )

        plotter.add_point_labels(
            pt,
            [s.kind],   # label text list
            text_color="white",
            point_color=None,
            font_size=18,
            shape=None,
            always_visible=True,
            shadow=True,
            fill_shape=False,
        )

    # ----------------------------------------
    # Show
    # ----------------------------------------
    plotter.show()

In [28]:
plot_velocity_field(vx, vy, vz, L, specs) # forward --> attractors

      !!!No serializer for vtkActor2D with id 00007fde4e883a80
      !!!No serializer for vtkActor2D with id 00007fde4e89bfb0
      !!!No serializer for vtkActor2D with id 00007fde4e8b4720
      !!!No serializer for vtkActor2D with id 00007fde4e8ccc50
      !!!No serializer for vtkActor2D with id 00007fde4e8e5180
      !!!No serializer for vtkActor2D with id 00007fde4e883a80
      !!!No serializer for vtkActor2D with id 00007fde4e89bfb0
      !!!No serializer for vtkActor2D with id 00007fde4e8b4720
      !!!No serializer for vtkActor2D with id 00007fde4e8ccc50
      !!!No serializer for vtkActor2D with id 00007fde4e8e5180
      !!!No serializer for vtkActor2D with id 00007fde4e883a80
      !!!No serializer for vtkActor2D with id 00007fde4e89bfb0
      !!!No serializer for vtkActor2D with id 00007fde4e8b4720
      !!!No serializer for vtkActor2D with id 00007fde4e8ccc50
      !!!No serializer for vtkActor2D with id 00007fde4e8e5180
      !!!No serializer for vtkActor2D with id 00007fde4

Widget(value="<iframe src='http://localhost:60958/index.html?ui=P_0x7fdf574cdeb0_13&reconnect=auto' style='wid…

In [ ]:
plot_velocity_field(vx, vy, vz, L, specs, integration_direction="backward")

      !!!No serializer for vtkActor2D with id 00007fde0e27f0c0
      !!!No serializer for vtkActor2D with id 00007fde0e2975f0
      !!!No serializer for vtkActor2D with id 00007fde0d2bc970
      !!!No serializer for vtkActor2D with id 00007fde0d2e7f00
      !!!No serializer for vtkActor2D with id 00007fde0e3044b0
      !!!No serializer for vtkActor2D with id 00007fde0e27f0c0
      !!!No serializer for vtkActor2D with id 00007fde0e2975f0
      !!!No serializer for vtkActor2D with id 00007fde0d2bc970
      !!!No serializer for vtkActor2D with id 00007fde0d2e7f00
      !!!No serializer for vtkActor2D with id 00007fde0e3044b0
      !!!No serializer for vtkActor2D with id 00007fde0e27f0c0
      !!!No serializer for vtkActor2D with id 00007fde0e2975f0
      !!!No serializer for vtkActor2D with id 00007fde0d2bc970
      !!!No serializer for vtkActor2D with id 00007fde0d2e7f00
      !!!No serializer for vtkActor2D with id 00007fde0e3044b0
      !!!No serializer for vtkActor2D with id 00007fde0

Widget(value="<iframe src='http://localhost:60958/index.html?ui=P_0x7fdff38b1340_14&reconnect=auto' style='wid…

      !!!No serializer for vtkActor2D with id 00007fde4e883a80
      !!!No serializer for vtkActor2D with id 00007fde4e89bfb0
      !!!No serializer for vtkActor2D with id 00007fde4e8b4720
      !!!No serializer for vtkActor2D with id 00007fde4e8ccc50
      !!!No serializer for vtkActor2D with id 00007fde4e8e5180
      !!!No serializer for vtkActor2D with id 00007fde4e883a80
      !!!No serializer for vtkActor2D with id 00007fde4e89bfb0
      !!!No serializer for vtkActor2D with id 00007fde4e8b4720
      !!!No serializer for vtkActor2D with id 00007fde4e8ccc50
      !!!No serializer for vtkActor2D with id 00007fde4e8e5180
      !!!No serializer for vtkActor2D with id 00007fde4e883a80
      !!!No serializer for vtkActor2D with id 00007fde4e89bfb0
      !!!No serializer for vtkActor2D with id 00007fde4e8b4720
      !!!No serializer for vtkActor2D with id 00007fde4e8ccc50
      !!!No serializer for vtkActor2D with id 00007fde4e8e5180
      !!!No serializer for vtkActor2D with id 00007fde4